<a href="https://colab.research.google.com/github/AlexandreXavierMartins/Sim/blob/main/simpy_aula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpy

Relógio da simulação:

a) Tarefas sequenciais

b) Tarefas paralelas

In [ ]:
import simpy
import numpy as np

np.random.seed(0)

def processo_seq(env):
  while True:

    yield env.timeout(1)
    print('Processo lento %f' %(env.now))
    yield env.timeout(0.2)
    print('Processo rápido %f' %(env.now))

env = simpy.Environment()
env.process(processo_seq(env))
env.run(until=5)

In [ ]:
import simpy
import numpy as np

np.random.seed(0)

def processo1(env):
  while True:
    yield env.timeout(1)
    print('Processo lento %f' %(env.now))
def processo2(env):
  while True:
    yield env.timeout(0.2)
    print('Processo rápido %f' %(env.now))


env = simpy.Environment()
env.process(processo1(env))
env.process(processo2(env))
env.run(until=5)

Criando os processos de uma entidade:

In [ ]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env):
  while True:
    print('Escavadeira trabalhando em %f' %(env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Começou manutenção da escavadeira em %f' %(env.now))
    repair = np.random.exponential(1)
    yield env.timeout(repair)

env = simpy.Environment()
env.process(escavadeira(env))


env.run(until=50)

Trabalhando com mais de uma entidade do mesmo tipo

In [ ]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env, nome):
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
    repair = np.random.exponential(1)
    yield env.timeout(repair)

env = simpy.Environment()
env.process(escavadeira(env, '%d' % 1))


env.run(until=50)

Trabalhando com recursos com capacidade limitada

In [ ]:
import simpy
import numpy as np

np.random.seed(0)


def escavadeira(env, nome, eqp):
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Escavadeira %s quebrou em %f' %(nome, env.now))
    with eqp.request() as req:
      yield req
      print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
      repair = np.random.exponential(1)
      yield env.timeout(repair)

env = simpy.Environment()
eqp = simpy.Resource(env, capacity=1)
for i in range(7):
  env.process(escavadeira(env, '%d' % i, eqp))



env.run(until=50)


Pegando o tempo de ocupação do recurso:

In [ ]:
import simpy
import numpy as np

np.random.seed(0)

ocp = 0
def escavadeira(env, nome, eqp):
  global ocp
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Escavadeira %s quebrou em %f' %(nome, env.now))


    with eqp.request() as req:
      yield req
      print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
      repair = np.random.exponential(1)
      ocp += repair
      yield env.timeout(repair)

env = simpy.Environment()
eqp = simpy.Resource(env, capacity=1)
for i in range(7):
  env.process(escavadeira(env, '%d' % i, eqp))



env.run(until=10)
print(f"Tempo total que o recurso 'eqp' ficou ocupado: {ocp}")


Tempo de Fila das Escavadeiras

In [ ]:
import simpy
import numpy as np

np.random.seed(0)

ocp = 0
tempo_total_fila = 0
cont_na_fila = 0

def escavadeira(env, nome, eqp):
  global ocp
  global tempo_total_fila
  global cont_na_fila
  while True:
    print('Escavadeira %s trabalhando em %f' %(nome, env.now))
    work = np.random.exponential(7)
    yield env.timeout(work)
    print('Escavadeira %s quebrou em %f' %(nome, env.now))
    tempo = env.now
    with eqp.request() as req:
      yield req
      print('Começou manutenção da escavadeira %s em %f' %(nome, env.now))
      tempo = env.now - tempo
      if tempo > t_max:
        t_max = tempo
      if tempo < t_min:
        t_min = tempo

      tempo_total_fila += tempo
      cont_na_fila += 1
      repair = np.random.exponential(1)
      ocp += repair
      yield env.timeout(repair)

env = simpy.Environment()
eqp = simpy.Resource(env, capacity=1)
for i in range(7):
  env.process(escavadeira(env, '%d' % i, eqp))



env.run(until=10)
print(f"Tempo total que o recurso 'eqp' ficou ocupado: {ocp}")
print(f"Tempo médio que uma escavadeira ficou na fila: {tempo_total_fila/cont_na_fila}")
print(f"Tempo máximo que uma escavadeira ficou na fila: {t_max}")
print(f"Tempo mínimo que uma escavadeira ficou na fila: {t_min}")


Exercício 1: Encontre o tempo máximo que uma escavadeira fica na fila.

Exercício 2: Encontre o número médio de escavadeiras na fila. (Obs. Para pegar o tamanho da fila em um determinado momento use o comando len(eqp.queue)

Exercício 3: Devido ao alto número de caminhões na fila do sistema da carregadeira o gerente decidiu fazer a simulação do sistema. Sendo assim o seu funcionamento pode ser descrito da seguinte forma: os caminhões são carregados pela FX215, gastando em média Normal(200, 15) segundos, depois seguem para o local onde está o britador com um tempo de Normal(90, 5) segundos, chegando lá eles descarregam com um tempo de Normal(50, 7) segundos e voltam para o local onde está a carregadeira com um tempo de Normal(70, 6) segundos. O sistema possui 8 caminhões. Simule por 7 dias, 24 horas por dia. Colete o tempo de fila na carregadeira e no britador.